In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.applications import InceptionResNetV2 #ImageNet
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras import optimizers
from kaggle_datasets import KaggleDatasets
# from keras.applications import VGG19 #ImageNet
# from keras.applications.vgg19 import preprocess_input
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.applications import ResNet50


Using TensorFlow backend.


In [2]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [4]:
# MY_GCS_PATH=KaggleDatasets().get_gcs_path('productdetection')


In [5]:
#Parameters
BATCH_SIZE = 16 
IMAGE_SIZE = 299
LEARNING_RATE = 3e-5 
CUT_OFF = 695
TRAINING_SIZE = 79060
VALID_SIZE = 26332
model_path = 'inc_resnet_%sx%s_v1.h5' % (IMAGE_SIZE, IMAGE_SIZE)

In [6]:

# imagegen = ImageDataGenerator(rescale=1./255,
#             shear_range=0.2,
#             zoom_range=0.2,
#             horizontal_flip=True,
#             validation_split=0.2)
imagegen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip = True, validation_split = 0.25)
train_gen = imagegen.flow_from_directory('../input/productdetection/dataset/dataset/train/train', 
                                         target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                         batch_size=BATCH_SIZE,
                                         class_mode='categorical',
                                         subset='training')
valid_gen = imagegen.flow_from_directory('../input/productdetection/dataset/dataset/train/train', 
                                         target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                         batch_size=BATCH_SIZE,
                                         class_mode='categorical',
                                         subset='validation')

Found 79060 images belonging to 42 classes.
Found 26332 images belonging to 42 classes.


In [7]:
# #Baseline Model
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  strides=2,
#                  activation='relu',
#                  input_shape=(256, 256, 3)))
# model.add(Dropout(0.5))
# model.add(Conv2D(32, kernel_size=(3, 3), strides=2, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(42, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimiser = RMSprop(lr=learning_rate),
#               metrics=[keras.metrics.TopKCategoricalAccuracy(k=1)])

In [8]:
# model.fit_generator(
#         train_gen,
#         steps_per_epoch=2000 // BATCH_SIZE,
#         epochs=4,
#         validation_data=valid_gen,
#         validation_steps=800 // BATCH_SIZE)


In [9]:
checkpoint = ModelCheckpoint('inc_resnet_post_v2.h5', monitor='val_loss', mode='min', save_best_only=True, save_weights_only=False)
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, 
                   restore_best_weights=True, verbose=1)

callback_list = [checkpoint, es]

opt = optimizers.Adam(learning_rate=LEARNING_RATE)

In [10]:
# #ImageNet Model

# ir_model = keras.applications.InceptionResNetV2(
#             include_top=False, 
#             weights='imagenet',
#             input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
            
# model = Sequential()
# model.add(ir_model)
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(42, activation='softmax', name='Output'))
# for cnn_block_layer in model.layers[0].layers:
#     cnn_block_layer.trainable = False
# model.layers[0].trainable = False

# model.compile(loss='categorical_crossentropy',
#                   optimizer = opt,
#                   metrics=[keras.metrics.TopKCategoricalAccuracy(k=1)])
# model.fit_generator(
#             train_gen,
#             steps_per_epoch=2000 // BATCH_SIZE,
#             epochs=25,
#             callbacks = callback_list,
#             validation_data=valid_gen,
#             validation_steps=800 // BATCH_SIZE)


In [11]:

baseModel = InceptionResNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(42, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

 

219062272/219055592 [==============================] - 2s 0us/step


In [12]:

print("[INFO] compiling model...")
model.compile(loss='categorical_crossentropy',
                optimizer = opt,
                metrics=[keras.metrics.TopKCategoricalAccuracy(k=1)])


[INFO] compiling model...


In [13]:
print("[INFO] training head...")
model.fit(train_gen,
            steps_per_epoch=12000 // BATCH_SIZE,
            epochs=30,
            validation_data=valid_gen,
            validation_steps=4200 // BATCH_SIZE)

[INFO] training head...
Epoch 1/30
750/750 [==============================] - 413s 550ms/step - loss: 1.7805 - top_k_categorical_accuracy: 0.4496 - val_loss: 1.4643 - val_top_k_categorical_accuracy: 0.5537
Epoch 2/30
750/750 [==============================] - 415s 553ms/step - loss: 1.3341 - top_k_categorical_accuracy: 0.5812 - val_loss: 1.2024 - val_top_k_categorical_accuracy: 0.6016
Epoch 3/30
750/750 [==============================] - 418s 557ms/step - loss: 1.2407 - top_k_categorical_accuracy: 0.6158 - val_loss: 1.1915 - val_top_k_categorical_accuracy: 0.6267
Epoch 4/30
750/750 [==============================] - 416s 554ms/step - loss: 1.1761 - top_k_categorical_accuracy: 0.6355 - val_loss: 1.1561 - val_top_k_categorical_accuracy: 0.6423
Epoch 5/30
750/750 [==============================] - 411s 548ms/step - loss: 1.1499 - top_k_categorical_accuracy: 0.6481 - val_loss: 1.1929 - val_top_k_categorical_accuracy: 0.6527
Epoch 6/30
750/750 [==============================] - 411s 548ms/s

In [14]:
train_gen.reset()
valid_gen.reset()
for layer in baseModel.layers[:CUT_OFF]:
    layer.trainable = False
for layer in baseModel.layers[CUT_OFF:]:
    layer.trainable = True
for layer in baseModel.layers:
    print("{}: {}".format(layer, layer.trainable))

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f4619753b50>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f461b1ebfd0>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f461b1f9e10>: False
<tensorflow.python.keras.layers.core.Activation object at 0x7f461b1a3610>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f461b1b4dd0>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f461aebce10>: False
<tensorflow.python.keras.layers.core.Activation object at 0x7f461aecbed0>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f461aed1050>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f461ae7ce50>: False
<tensorflow.python.keras.layers.core.Activation object at 0x7f461ae7cb10>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f461ae82ed0>: False
<tensorflow.python.ker

In [15]:
print("[INFO] re-compiling model...")
model.compile(loss='categorical_crossentropy',
                  optimizer = opt,
                  metrics=[keras.metrics.TopKCategoricalAccuracy(k=1)])
print("[INFO] re-training head...")
model.fit(train_gen,
            steps_per_epoch=12000 // BATCH_SIZE,
            epochs=25,
            validation_data=valid_gen,
            callbacks = callback_list,
            validation_steps=4200 // BATCH_SIZE)

[INFO] re-compiling model...
[INFO] re-training head...
Epoch 1/25
750/750 [==============================] - 393s 524ms/step - loss: 0.6619 - top_k_categorical_accuracy: 0.8021 - val_loss: 1.0290 - val_top_k_categorical_accuracy: 0.8001
Epoch 2/25
750/750 [==============================] - 392s 523ms/step - loss: 0.5700 - top_k_categorical_accuracy: 0.8038 - val_loss: 1.0252 - val_top_k_categorical_accuracy: 0.8072
Epoch 3/25
750/750 [==============================] - 394s 525ms/step - loss: 0.5243 - top_k_categorical_accuracy: 0.8100 - val_loss: 1.0264 - val_top_k_categorical_accuracy: 0.8123
Epoch 4/25
750/750 [==============================] - 394s 525ms/step - loss: 0.4796 - top_k_categorical_accuracy: 0.8147 - val_loss: 0.9473 - val_top_k_categorical_accuracy: 0.8171
Epoch 5/25
750/750 [==============================] - 392s 523ms/step - loss: 0.4782 - top_k_categorical_accuracy: 0.8189 - val_loss: 1.0240 - val_top_k_categorical_accuracy: 0.8203
Epoch 6/25
750/750 [==============

In [16]:
model.save("inc_resnet_saved_v2.h5")
best_model = load_model('../input/productdetection/inc_resnet_post_v1.h5')

In [17]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    directory="../input/productdetection/dataset/dataset/test",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode=None,
    shuffle=False)


In [18]:
predictions = best_model.predict_generator(test_generator, verbose = 1, steps = 12192//BATCH_SIZE)


In [19]:
predicted_class_indices=np.argmax(predictions,axis=1)
labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred_final = [labels[k] for k in predicted_class_indices]

In [20]:
filenames=test_generator.filenames
results=pd.DataFrame({"filename":filenames,
                      "category":pred_final})

In [21]:
results['filename'] = results['filename'].str[5:]

In [22]:
results.set_index('filename')
results.to_csv("submissionv1.csv")